# Direct Distances

In this approach, I make the assumption that the distance between two shops is independent of the road network.

## Generate Random Warehouses

In [1]:
import copy
import os
import pickle
import random

from geopy.geocoders import Nominatim
from haversine import Unit, haversine_vector
from pulp import *

In [2]:
if not os.path.exists("warehouses.pkl"):
    print("Generating warehouses info...")

    warehouses = []
    for i in range(1, 101):
        name = f"Warehouse {i}"
        lat = random.uniform(
            10.668799, 10.878041
        )  # Latitude range of Ho Chi Minh City
        long = random.uniform(
            106.451840, 106.813580
        )  # Longitude range of Ho Chi Minh City
        description = None
        warehouses.append(
            {
                "index": i,
                "name": name,
                "lat": lat,
                "long": long,
                "description": "",
            }
        )

    geolocator = Nominatim(user_agent="bk-imp")

    for i in range(len(warehouses)):
        lat, long = warehouses[i]["lat"], warehouses[i]["long"]
        location = geolocator.reverse(f"{lat}, {long}")
        address = location.address
        warehouses[i]["description"] = address

    with open("warehouses.pkl", "wb") as f:
        # Shuffle warehouses to generate random order
        random.shuffle(warehouses)
        pickle.dump(warehouses, f)
else:
    with open("warehouses.pkl", "rb") as f:
        warehouses = pickle.load(f)

In [3]:
with open("stores.pkl", "rb") as f:
    stores = pickle.load(f)

## Down Sample

In [5]:
warehouses = random.choices(warehouses, k=10)
stores = random.choices(stores, k=100)

## Calculate Distance Matrix

### Between Stores

In [6]:
stores_dinstance_matrix = haversine_vector(
    [(store["lat"], store["long"]) for store in stores],
    [(store["lat"], store["long"]) for store in stores],
    Unit.KILOMETERS,
    comb=True,
)

In [7]:
stores_dinstance_matrix.shape

(100, 100)

### Between Warehouses vs Stores

In [8]:
warehouses_stores_dinstance_matrix = haversine_vector(
    [(store["lat"], store["long"]) for store in stores],
    [(warehouse["lat"], warehouse["long"]) for warehouse in warehouses],
    Unit.KILOMETERS,
    comb=True,
)

In [9]:
warehouses_stores_dinstance_matrix.shape

(10, 100)

## Linear Programming

In [10]:
# Set up the problem
problem = LpProblem("Warehouse_Optimization", LpMinimize)

### Define the decision variables

In [11]:
M = 10  # Number of warehouses
N = 100 # Number of stores
warehouses = range(M)
f = random.sample(range(100, 201), M)  # Operational cost of warehouses

stores = range(N)

cs = copy.copy(stores_dinstance_matrix)  # Delivery cost between stores
cw = copy.copy(
    warehouses_stores_dinstance_matrix
)  # Delivery cost between warehouses vs stores

In [12]:
x = LpVariable.dicts("warehouse_selection", warehouses, cat="Binary")
r = LpVariable.dicts("store_assignment", (stores, stores), cat="Binary")
wr = LpVariable.dicts(
    "warehouse_store_connection", (warehouses, stores), cat="Binary"
)

### Define the objective function

In [13]:
total_cost = (
    lpSum(f[i] * x[i] for i in warehouses)
    + lpSum(cs[i][j] * r[i][j] for i in stores for j in stores)
    + lpSum(cw[i][j] * wr[i][j] for i in warehouses for j in stores)
)
problem += total_cost

### Define the constraints

In [14]:
# Each store must be assigned to exactly one route (warehouse)
for i in stores:
    problem += lpSum(r[i][j] for j in stores) == 1

In [15]:
# Limit the number of warehouses to be selected
problem += lpSum(x[i] for i in warehouses) <= M

In [17]:
# Each cyclic route must have one first store, one last store, and consecutive segments
for i in warehouses:
    problem += lpSum(wr[i][k] for k in stores) == 2
    for m in stores:
        for n in stores:
            problem += lpSum(r[m][k] + wr[i][m] for k in stores if r[m][n] == 1) == 1
            problem += lpSum(r[n][k] + wr[i][n] for k in stores if r[m][n] == 1) == 1

## Solve the problem!!!

In [18]:
problem.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/terrabot/bk-imp/.venv/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/dd71c4b20ce947c5a2932132775bec2e-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/dd71c4b20ce947c5a2932132775bec2e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 200116 COLUMNS
At line 20444045 RHS
At line 20644157 BOUNDS
At line 20655168 ENDATA
Problem MODEL has 200111 rows, 11010 columns and 20211010 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 20.88 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       27.27   (Wallclock seconds):       28.56



-1

In [19]:
# Print the solution
print("Optimal warehouses:")
for i in warehouses:
    if x[i].varValue == 1:
        print("Warehouse", i)

print("Optimal routes:")
for i in stores:
    for j in stores:
        if r[i][j].varValue == 1:
            print("Store", i, "-> Store", j)

print("Optimal warehouse-store connections:")
for i in warehouses:
    for j in stores:
        if wr[i][j].varValue == 1:
            print("Warehouse", i, "-> Store", j)

print("Total cost:", value(problem.objective))

Optimal warehouses:
Optimal routes:
Store 0 -> Store 0
Store 1 -> Store 1
Store 2 -> Store 2
Store 3 -> Store 3
Store 4 -> Store 4
Store 5 -> Store 5
Store 6 -> Store 1
Store 7 -> Store 7
Store 8 -> Store 8
Store 9 -> Store 9
Store 10 -> Store 10
Store 11 -> Store 11
Store 12 -> Store 12
Store 13 -> Store 13
Store 14 -> Store 30
Store 15 -> Store 15
Store 16 -> Store 16
Store 17 -> Store 17
Store 18 -> Store 18
Store 19 -> Store 19
Store 20 -> Store 20
Store 21 -> Store 21
Store 22 -> Store 22
Store 23 -> Store 23
Store 24 -> Store 24
Store 25 -> Store 25
Store 26 -> Store 26
Store 27 -> Store 23
Store 28 -> Store 28
Store 29 -> Store 29
Store 30 -> Store 14
Store 31 -> Store 31
Store 32 -> Store 32
Store 33 -> Store 33
Store 34 -> Store 34
Store 35 -> Store 35
Store 36 -> Store 36
Store 37 -> Store 37
Store 38 -> Store 38
Store 39 -> Store 39
Store 40 -> Store 40
Store 41 -> Store 41
Store 42 -> Store 42
Store 43 -> Store 43
Store 44 -> Store 44
Store 45 -> Store 45
Store 46 -> Store 